# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict
# Import API key
from api_keys import geoapify_key
url = "https://api.geoapify.com/v2/places?categories=commercial.supermarket&filter=rect%3A10.716463143326969%2C48.755151258420966%2C10.835314015356737%2C48.680903341613316&limit=20&apiKey=86b6de2b14a045168e86bd10f69745f1"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tura,25.5198,90.2201,23.01,36,65,1.69,IN,1712795363
1,1,bethel,41.3712,-73.4140,14.93,78,100,2.57,US,1712795099
2,2,kerikeri,-35.2268,173.9474,21.06,92,100,7.46,NZ,1712795363
3,3,papao,-17.7500,-149.5500,30.82,83,89,8.93,PF,1712795236
4,4,utrik,11.2278,169.8474,26.74,78,99,10.06,MH,1712795364


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo=True,
                                        tiles = "OSM",
                                        scale = "Humidity",
                                        frame_width = 700,
                                        frame_height = 500,
                                        hover_cols=['City','Humidity','Country'],
                                        c = 'City')
                                        #size = "Humidity")
# YOUR CODE HERE

# Display the map
map_plot_1
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
ideal_condition_df = city_data_df
ideal_condition_df = ideal_condition_df.loc[(ideal_condition_df["Max Temp"] < 27) & (ideal_condition_df["Max Temp"] > 21)]
ideal_condition_df = ideal_condition_df.loc[(ideal_condition_df["Wind Speed"] < 4.5)]
ideal_condition_df = ideal_condition_df.loc[(ideal_condition_df["Cloudiness"] == 0)]
# YOUR CODE HERE

# Drop any rows with null values
ideal_condition_df = ideal_condition_df.dropna()
# YOUR CODE HERE

# Display sample data
ideal_condition_df
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
209,209,maceio,-9.6658,-35.7353,25.69,88,0,2.06,BR,1712795383
235,235,rosarito,32.3333,-117.0333,26.80,59,0,3.99,MX,1712795386
272,272,saint-pierre,-21.3393,55.4781,24.82,69,0,3.60,RE,1712795388
376,376,pacific grove,36.6177,-121.9166,24.25,75,0,2.06,US,1712795395
442,442,yanbu,24.0891,38.0637,22.02,37,0,3.60,SA,1712795404
462,462,tannum sands,-23.9504,151.3693,24.94,38,0,3.09,AU,1712795407
519,519,yreka,41.7354,-122.6345,23.10,27,0,0.00,US,1712795411
557,557,mhamid,29.8200,-5.7200,21.25,16,0,2.16,MA,1712795414


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_condition_df[["City","Country","Lat","Lng","Humidity"]].copy()
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# YOUR CODE HERE

# Display sample data
hotel_df
# YOUR CODE HERE

,City,Country,Lat,Lng,Humidity,Hotel Name
209,maceio,BR,-9.6658,-35.7353,88,
235,rosarito,MX,32.3333,-117.0333,59,
272,saint-pierre,RE,-21.3393,55.4781,69,
376,pacific grove,US,36.6177,-121.9166,75,
442,yanbu,SA,24.0891,38.0637,37,
462,tannum sands,AU,-23.9504,151.3693,38,
519,yreka,US,41.7354,-122.6345,27,
557,mhamid,MA,29.8200,-5.7200,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"
radius = 10000 # YOUR CODE HERE
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "filter":"",
    "bias" :"",
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row["Lng"]},{row["Lat"]},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{row["Lng"]},{row["Lat"]}" # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    
    name_address = requests.get(base_url,params=params,headers=headers)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
maceio - nearest hotel: Maceió Hostel e Pousada
rosarito - nearest hotel: WorldMark La Paloma
saint-pierre - nearest hotel: Tropic Hotel
pacific grove - nearest hotel: Pacific Grove Inn
yanbu - nearest hotel: No hotel found
tannum sands - nearest hotel: No hotel found
yreka - nearest hotel: Best Western Minner's Inn
mhamid - nearest hotel: Hotel Kasbah Azalay


,City,Country,Lat,Lng,Humidity,Hotel Name
209,maceio,BR,-9.6658,-35.7353,88,Maceió Hostel e Pousada
235,rosarito,MX,32.3333,-117.0333,59,WorldMark La Paloma
272,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
376,pacific grove,US,36.6177,-121.9166,75,Pacific Grove Inn
442,yanbu,SA,24.0891,38.0637,37,No hotel found
462,tannum sands,AU,-23.9504,151.3693,38,No hotel found
519,yreka,US,41.7354,-122.6345,27,Best Western Minner's Inn
557,mhamid,MA,29.8200,-5.7200,16,Hotel Kasbah Azalay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo=True,
                                    tiles = "OSM",
                                    #scale = "Humidity",
                                    hover_cols=['Hotel Name', 'City','Humidity','Country',],
                                    frame_width = 700,
                                    frame_height = 500,
                                    c = 'City')
                                        #size = "Humidity")

# YOUR CODE HERE

map_plot_2
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Humidity,Country)